In [1]:
import numpy as np
import pandas as pd

In [ ]:
# load data
df = pd.read_excel(r'your path/filename.xlsx')

In [3]:
# define relevant columns  = part number, production supplier,  destribution supplier + 
rel_cols = list(df.columns[[27, 3, 5]]) + list(df.columns[44:49]) + list(df.columns[50:52])

# define columns for relevant infos from colleagues
rel_cols_info_lis = list(df.columns[[27]]) + list(df.columns[44:49]) + list(df.columns[50:52])

# create a df with the relevant columns
df2 = df[rel_cols].astype(str).replace(np.nan, '')

# concatenate a string for identification purposes
df['concatenate'] = df2[rel_cols].agg('-'.join, axis = 1)
df['concatenate infos LI'] = df2[rel_cols_info_lis].agg('-'.join, axis = 1)

# list all relavent part numbers that occur muliple times 
snr_unique_gr_2 = df[(df['# part numbers']>1)
                     & (df.iloc[:, 53].str.contains('EMM'))
                     | (df.iloc[:, 53].str.contains('EMO'))
                    ]['part numbers'].unique()

# iterate through list, and check if there is only one prod-supplier / dist supplier for each partnumber
# if not, check, if there is more than one row for a part number and for one supplier
df['entry mistake'] = np.nan
for snr in snr_unique_gr_2:
    # df for each part number
    df_check = df[df['part number'] == snr][rel_cols]
    
    anz_pl = len(set(df_check['Prod.-Supplier'].to_list()))
    anz_bl = len(set(df_check['Dist.-Supplier'].to_list()))
    if anz_pl == 1 and anz_bl == 1:
        anz = len(set(df[df['part number'] == snr]['concatenate'].to_list()))
        if anz > 1:
            df.loc[df['part number'] == snr, 'entry mistake'] = 'x'
    else:
        # list of all prod suppliers / dist suppliers for a part number
        lst_pl = list(set(df[df['part number'] == snr]['Prod.-Supplier'].to_list()))
        lst_bl = list(set(df[df['part number'] == snr]['Dist.-Supplier'].to_list()))
        
        for l in lst_pl:
            df_pl = list(set(df[(df['part number'] == snr) & (df['Prod.-Supplier'] == l)]['concatenate infos LI'].to_list()))
            if len(df_pl)> 1:
                df.loc[df['part number'] == snr, 'entry mistake'] = 'x'
        
        for l in lst_bl:
            df_bl = list(set(df[(df['part number'] == snr) & (df['Dist.-Supplier'] == l)]['concatenate infos LI'].to_list()))
            if len(df_bl)> 1:
                df.loc[df['part number'] == snr, 'entry mistake'] = 'x'

In [4]:
# return all the part numbers that have entry mistakes in the list
df[df['entry mistake'].notna()][rel_cols + ['entry mistake']]['part number'].unique()

array([9443161, 7921852, 1541420, 1541423, 1541841, 1541842, 1541843,
       9829276, 9445666, 9445667, 1539644, 9446868, 9446869, 9829296,
       8536349, 8536350, 8536347, 8536348, 8536351, 9829772, 9829773],
      dtype=object)

In [ ]:
# create excel file
df.to_excel(r'your path/filename.xlsx')